In [159]:
# Module imports
import copy
import datetime
import os
import pickle

import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = 'raise'

VERSION = 0.8


# Need to define in main or we can't pickle the data objects
class DataFrames(object):
    def __init__(self):
        self.catalog = None
        self.ftir = None
        self.reagent = None
        self.mla = None
        self.hr = None
        self.combined = None


def now():
    return datetime.datetime.now().strftime("%d/%m/%y %H:%M:%S")


def get_rename_columns_map():
    sheet_id = '1CgqTjdKizat-g7K7-AAuVIazQFKJ3WAAPHR-Qpa49lU'
    ss_range = 'ColumnMap!A:B'
    result = GSHEETS_SERVICE.spreadsheets().values().get(spreadsheetId=sheet_id,
                                                         range=ss_range).execute()
    values = result.get('values', [])
    assert values[0] == ['OriginalColumn', 'CanonicalColumn'], values[0]
    return { cm[0] : cm[1] for cm in values[1:] if len(cm) >= 2 }


def gsheets_service():
    from googleapiclient.discovery import build
    from httplib2 import Http
    from oauth2client import file, client, tools
    # If modifying these scopes, delete the file token.json.
    #Ensure that the creds file is always taken from the current working folder
        #This allows two people on different PCs to merge changes more easily.
    CREDS_FILE = os.path.join(os.path.realpath('./'),'JensDataExportJupyter_client_secret.json')
    SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        import argparse
        flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args([])
        flow = client.flow_from_clientsecrets(CREDS_FILE, SCOPES)
        creds = tools.run_flow(flow, store, flags)
    service = build('sheets', 'v4', http=creds.authorize(Http()))
    return service


def get_data(spreadsheet_id):
    catalog_range = 'Catalog!A:R'
    ftir_range = 'FTIR!A:X'
    reagent_range = 'Reagent!A:W'
    mla_range = 'MLA!A:R'
    hr_range = 'Interventions!A:BJ'
    
    df_catalog = get_df(spreadsheet_id, catalog_range)
    df_catalog = canonicalise_df(df_catalog, source='catalog')
    df_ftir = get_df(spreadsheet_id, ftir_range)
    df_ftir = canonicalise_df(df_ftir, source='ftir')
    df_reagent = get_df(spreadsheet_id, reagent_range)
    df_reagent = canonicalise_df(df_reagent, source='reagent')
    df_mla = get_df(spreadsheet_id, mla_range, mla=True)
    df_mla = canonicalise_df(df_mla, source='mla')
    try:
        df_hr = get_df(spreadsheet_id, hr_range)
    except ValueError:
        df_hr = None
    if df_hr is not None:
        pass
        df_hr = canonicalise_df(df_hr, source='hr')

    df = DataFrames()
    df.catalog = df_catalog
    df.ftir = df_ftir
    df.reagent = df_reagent
    df.mla = df_mla
    df.hr = df_hr
    
    return df


def get_df(spreadsheet_id, ss_range, mla=False):
    # Call the Sheets API
    result = GSHEETS_SERVICE.spreadsheets().values().get(spreadsheetId=spreadsheet_id,
                                                         range=ss_range).execute()
    values = result.get('values', [])
    if not values:
        print('*** No data found ***')
        return None

    # mla has irrelevant stuff in columns 1 and 3 and sample numbers in first column
    if mla:
        values.pop(0)
        values.pop(1)
        def not_blank(row):
            return len(row[0]) > 0       
    else:
        def not_blank(row):
            return sum(map(len, row[:6])) > 0

    rows = list(filter(not_blank, values))
    if not rows:
        print('*** No data found after pruning rows! ***')
        return None
    
    columns = enumerate_duplicates(rows[0])
    ncols = len(rows[0])
    row_max = max(map(len, rows[1:]))
    width = min(ncols, row_max)
    return pd.DataFrame(rows[1:], columns=columns[:width])


def enumerate_duplicates(row):
    """Append a counter to duplicate labels"""
    SEPARATOR = '.'
    duplicates = {}
    updated_row = []
    for r in row:
        count = duplicates.get(r, 0)
        if count > 0:
            label = "{}{}{}".format(r, SEPARATOR, count)
        else:
            label = r
        updated_row.append(label)
        duplicates[r] = count + 1
    return updated_row


def canonicalise_df(df, source=None):
    """Initial cleaning of all dataframes"""
    if source:
        print("Canonicalising %s" % source)
    # Standardise names
    df.rename(columns=RENAME_COLUMN_MAP, inplace=True)
    def fix_timestamp(x):
        return pd.to_datetime(str(x), format='%d/%m/%Y %H:%M:%S')
    if 'Timestamp' in df.columns:
        df.loc[:, 'Timestamp'] = df['Timestamp'].map(fix_timestamp)
    #df.loc[:, 'SampleNumber'] = df['SampleNumber'].apply(fix_sample_number)
    df.dropna(subset=['SampleNumber'], inplace=True)
    if set(df.columns.values).intersection(set([np.nan, ''])):
        raise RuntimeError("Blank column names in Dataframe")
    return df


def check_individual_dataframes(dfs):
    errors = []
    for name in ['catalog', 'ftir', 'reagent', 'mla', 'hr']:
        df = getattr(dfs, name)
        error = _check_individual_dataframe(df, name)
        errors.append(error)
    if any(errors):
#         raise RuntimeError("Data errors need to be fixed!")
        print("Data errors need to be fixed!")

        
def _check_individual_dataframe(df, source):
    cname = 'SampleNumber'
    dups = df[cname].duplicated()
    if any(dups):
        NSTAR = 80
        print("*" * NSTAR)
        print("WARNING!! %s sheet has duplicate sample numbers: %s" % (source, df[cname][dups].values))
        print("*" * NSTAR)
        return True
    return False


def person_id_from_samplenumber(x):
    if isinstance(x, float) and np.isnan(x):
        return x # leave NaN's alone
    if (isinstance(x, str) or isinstance(x, unicode)) and len(x) == 0:
        return np.nan
    if len(x) != 5:
        return np.nan
    try:
        pn = 'P{:04d}'.format(int(x[-3:]))
    except ValueError:
        print("!!! Bad ID \'%s\'" % pn)
        pn = x
    return pn


def add_person_id(hr_df):
    """Create the unique PID column"""
    hr_df['PID'] = dfs.hr['SampleNumber']
    # Copy over SampleNumbers from Previous_sample
    mask = hr_df['Previous_sample'].isnull()
    hr_df['PID'] = hr_df['PID'].where(mask, hr_df['Previous_sample'])
    # Clean up values
    hr_df['PID'] = hr_df['PID'].apply(person_id_from_samplenumber)
    return hr_df


def merge_ftir_drug_columns(df):
    # Copy over 'Other' substances into the main column
    target_label = 'Substance detected'
    source_label = 'Compound detected'
    to_drop = [source_label, 'Hit Confidence.1']
    other_mask = ~df[target_label].str.startswith('Other').fillna(value=False)
    df[target_label].where(other_mask, df[source_label], inplace=True) # Copy values from source_label column over
    df.drop(to_drop, axis=1, inplace=True) # Remove now redundant columns
    df.rename(columns={target_label : 'Primary_hit', 'Hit Confidence' : 'Primary_confidence'}, inplace=True) # Rename Columns

    # Column names appear to be reversed - compound now is substance!!
    target_label = 'Compound detected (Subtraction)'
    source_label = 'Substance detected.1'
    to_drop = [source_label, 'Hit Confidence.3']
    other_mask = ~df[target_label].str.startswith('Other').fillna(value=False)
    df[target_label].where(other_mask, df[source_label], inplace=True) # Copy values from source_label column over
    df.drop(to_drop, axis=1, inplace=True) # Remove now redundant columns
    df.rename(columns={target_label : 'Secondary_hit', 'Hit Confidence.2' : 'Secondary_confidence'}, inplace=True) # Rename Columns

    
def calculate_final_results(df):
    """Calculate final result"""
    # Where 'ftir_Substance(s) detected' is null we use the ftir_Primary_hit
    mask = ~df['ftir_Substance(s) detected'].isin(['', np.nan, None])
    df['ftir_Substance(s) detected'] = df['ftir_Substance(s) detected'].where(mask, df['ftir_Primary_hit'])
    # Find where'reagent_Substance(s) detected' contains anything but 'No active component identified'
    mask = df['reagent_Substance(s) detected'].isin([None, np.nan,'No active component identified' ])
    # Default is 'ftir_Substance(s) detected'
    df['Final_result_calculated'] = df['ftir_Substance(s) detected']
    # Copy over anything from 'reagent_Substance(s) detected'
    df['Final_result_calculated'] = df['Final_result_calculated'].where(mask, df['reagent_Substance(s) detected'])
    # Need to lowercase for comparison
    df['Final_result_calculated'] = df['Final_result_calculated'].astype(str).str.lower()
    df['catalog_SoldAs'] = df['catalog_SoldAs'].astype(str).str.lower()
    
    # Calculate where they do/don't match
    df['As_expected'] = (df['Final_result_calculated'] == df['catalog_SoldAs']).map({True : 'Yes', False : 'No'})

    # Guy 28/10/18: 'As_expected' should be null whenever the sample is found,
    # when the submission 'acquired as" data is blank or unknown, or when the sample is from Amnesty
    mask1 = df['catalog_SoldAs'].isin(['found', 'found or otherwise not known', np.nan, None])
    mask2 = df['catalog_SampleSource'] != 'Public'
    mask = mask1 | mask2
    # jmht - could check against: 'hr_Was the sample bought, given or found?
    df.loc[mask, ['As_expected']] = np.nan
    return df


def merge_dataframes(dfs):
    # Clean up ftir sheet
    merge_ftir_drug_columns(dfs.ftir)

    if dfs.hr is not None:
        dfs.hr = add_person_id(dfs.hr)
    
    # Rename columns to identify source dataframe
    dfs.catalog.columns = ['catalog_'+ name if name != 'SampleNumber' else name for name in dfs.catalog.columns]
    dfs.ftir.columns = ['ftir_'+ name if name != 'SampleNumber' else name for name in dfs.ftir.columns]
    dfs.mla.columns = ['mla_'+ name if name != 'SampleNumber' else name for name in dfs.mla.columns]
    dfs.reagent.columns = ['reagent_'+ name if name != 'SampleNumber' else name for name in dfs.reagent.columns]
    if dfs.hr is not None:
        dfs.hr.columns = ['hr_'+ name if name != 'SampleNumber' else name for name in dfs.hr.columns]

    # Remove all but the last of any duplicate SampleNumber
    remove_duplicates = False
    if remove_duplicates:
        mask = ~dfs.catalog['SampleNumber'].duplicated(keep=False) | ~dfs.catalog['SampleNumber'].duplicated(keep='last')
        dfs.catalog = dfs.catalog[mask]
        mask = ~dfs.ftir['SampleNumber'].duplicated(keep=False) | ~dfs.ftir['SampleNumber'].duplicated(keep='last')
        dfs.ftir = dfs.ftir[mask]
        mask = ~dfs.mla['SampleNumber'].duplicated(keep=False) | ~dfs.mla['SampleNumber'].duplicated(keep='last')
        dfs.mla = dfs.mla[mask]
        mask = ~dfs.reagent['SampleNumber'].duplicated(keep=False) | ~dfs.reagent['SampleNumber'].duplicated(keep='last')
        dfs.reagent = dfs.reagent[mask]
        if dfs.hr is not None:
            mask = ~dfs.hr['SampleNumber'].duplicated(keep=False) | ~dfs.hr['SampleNumber'].duplicated(keep='last')
            dfs.hr = dfs.hr[mask]

    # First outer join on catalog/ftir to make sure we collect all possible information - this will result in
    # some rows where there was no catalog data, only ftir data, but this is ok as when we merge with hr we will
    # throw away any row that doesn't have a corresponding sample number in HR. This was even if catalog data is
    # missing, we still get the FTIR data, which may be enough for our purposes
    df_all = pd.merge(dfs.catalog, dfs.ftir, how='outer', on=['SampleNumber'])
    # Add in mla data - only for when there are existing sample numbers
    df_all = pd.merge(df_all, dfs.mla, how='left', on=['SampleNumber'])
    df_all = pd.merge(df_all, dfs.reagent, how='left', on=['SampleNumber'])
    if dfs.hr is not None:
        # inner join -> merge only where there are matching sample numbers
        df_all = pd.merge(df_all, dfs.hr, how='inner', on=['SampleNumber'])
    dfs.combined = df_all


def add_final_data(dfs):
    # Calculate final result
    dfs.combined = calculate_final_results(dfs.combined)
    
    # Add unique columns
    #dfs.combined.insert(loc=0, column='Festival', value=festival)
    #dfs.combined.insert(loc=1, column='UID', value=dfs.combined[['Festival', 'SampleNumber']].apply(lambda x: '_'.join(x), axis=1))
    dfs.combined['Version'] = VERSION


def check_matching_values(cols):
    # Check all lower-case string values are the same
    return len(np.unique(cols.astype(str).str.strip().str.lower().values)) != 1


def check_merged_data(dfs):
    NSTAR = 80
    cols = ['catalog_SoldAs', 'ftir_SoldAs', 'hr_SoldAs']
    unmatched = dfs.combined[cols].apply(check_matching_values, axis=1)
    if any(unmatched):
        print("*" * NSTAR)
        print("Unmatched SoldAs data")
        print(dfs.combined.loc[unmatched, ['SampleNumber'] + cols])
        print("*" * NSTAR)


    cols = ['catalog_AlreadyTried', 'ftir_AlreadyTried', 'hr_AlreadyTried']
    unmatched = dfs.combined[cols].apply(check_matching_values, axis=1)
    if any(unmatched):
        print("*" * NSTAR)
        print("Unmatched AlreadyTried data")
        print(dfs.combined.loc[unmatched, ['SampleNumber'] + cols])
        print("*" * NSTAR)


In [160]:
SPREADSHEET_ID = '1LUFyELCP6VXH3rU8-RVa7NQvCQBYQXRsIoFDbokDd4s'

print("Script running from: %s" % os.path.realpath(os.getcwd()))
try:
    RENAME_COLUMN_MAP = get_rename_columns_map()
except ConnectionResetError as e:
    raise RuntimeError("Cannot connect to server to get RENAME_COLUMN_MAP: %s" % e)

GSHEETS_SERVICE = gsheets_service()
dfs = get_data(SPREADSHEET_ID)

# Run initial checks
check_individual_dataframes(dfs)

# Merge everything together
merge_dataframes(dfs)
add_final_data(dfs)

# Check merged data
check_merged_data(dfs)

print("Finished processing dataframes at %s" % now())

Script running from: /opt/random
Canonicalising catalog
Canonicalising ftir
Canonicalising reagent
Canonicalising mla
Canonicalising hr
********************************************************************************
Unmatched SoldAs data
  SampleNumber catalog_SoldAs ftir_SoldAs hr_SoldAs
1        F0077           mdma        MDMA     pill 
8        F0095            lsd         NaN    "Acid"
********************************************************************************
********************************************************************************
Unmatched AlreadyTried data
   SampleNumber catalog_AlreadyTried ftir_AlreadyTried hr_AlreadyTried
4         F0089                   No                No             Yes
8         F0095                  Yes               NaN             Yes
10        F0108                  Yes               Yes              No
12        F0120                   No               Yes             Yes
13        F0119                   No               Yes      

In [161]:
filename = 'LoopData_%s.xls' % VERSION
writer = pd.ExcelWriter(filename)
dfs.combined.to_excel(writer, 'MergedData', index=False)
writer.save()
print(now() + " Wrote version %s to file: %s" % (VERSION, filename))

15/12/18 17:32:52 Wrote version 0.8 to file: LoopData_0.8.xls
